# 카메라에 연결

In [1]:
import cv2
from google.colab.patches import cv2_imshow
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import torch
import numpy as np

In [2]:
# 파이썬 take_photo 함수
def take_photo(filename='photo.jpg', quality=0.8):

  # javascript 작성 시작
  js = Javascript('''
    async function takePhoto(quality) {

      //div(공간) 생성
      const div = document.createElement('div');

      //capture button 생성
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      // //quit button 생성
      // const quit = document.createElement('button');
      // quit.textContent = 'Quit';
      // div.appendChild(quit);
      // //quit button에 들어갈 함수 추가 - 코드 종류 함수
      // function winClose(){
      //   // window.open("about:black","_self").close();
      //   window.open("about:black","_self").exit();
      // }
      // //quit button에 함수 할당
      // quit.addEventListener('click', winClose);

      //video 생성
      const video = document.createElement('video');
      //비디오 모양 네모네모
      video.style.display = 'block';
      //카메라(웹캠) 불러오기
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      //div 밑에 child 공간 추가
      document.body.appendChild(div);
      //공간에 video 넣기
      div.appendChild(video);
      //video와 웹캠 연결
      video.srcObject = stream;
      //await -> 비동기식 처리 (thread와 관련)(async와 세트)
      // await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // // Wait for Capture to be clicked.
      // await new Promise((resolve) => capture.onclick = resolve);

      //canvas 생성
      const canvas = document.createElement('canvas');
      //크기 맞추기
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      //이미지 그리기
      canvas.getContext('2d').drawImage(video, 0, 0);
      //비디오 끄고, div 삭제 코드가 있어야 캡쳐 후 카메라가 여러 개 생기지 않음
      //비디오 끄기
      stream.getVideoTracks()[0].stop();
      //div 삭제
      div.remove();
      // 파일 주소 반환
      return canvas.toDataURL('image/jpeg', quality);

    }
    ''')
  display(js)

  # javascript 코드 실행한 결과 반환값을 파이썬에게 전달
  data = eval_js('takePhoto({})'.format(quality))

  #웹 브라우저에서 데이터를 저장할때 base64로 저장
  binary = b64decode(data.split(',')[1])

  with open(filename, 'wb') as f:
      f.write(binary)
  return filename

In [3]:
from importlib.util import module_for_loader
def detect_objects_yolov5(weights_path, img_size=416):
    # YOLOv5 모델 로드
    model = torch.hub.load('ultralytics/yolov5', 'custom', path=weights_path, force_reload=True)
    layer_names = model.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in module_for_loader.getUnconnectedOutLayers()]

    classes = []

    # 웹캠 비디오 스트림 열기 및 객체 검출 수행
#    while cv2.waitKey(33) != ord('q'):
    while True:

        try:

            # 카메라로부터 이미지 가져오기
            filename = take_photo()

            # 프레임 읽기
            frame = cv2.imread(filename)

            # # 프레임 전처리
            # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = torch.from_numpy(frame).float() / 255.0
            # img = img.permute(2, 0, 1)
            # img = img.unsqueeze(0)
            # img = img.to(model.model.device)

            # 객체 탐지
            # results = model(img)
            blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
            model.setInput(blob)
            outs = model.forward(output_layers)

            # Showing informations on the screen
            class_ids = []
            confidences = []
            boxes = []
            for out in outs:
                for detection in out:
                    scores = detection[5:]
                    class_id = np.argmax(scores)
                    confidence = scores[class_id]
                    if confidence > 0.5:
                        # Object detected
                        center_x = int(detection[0])
                        center_y = int(detection[1])
                        w = int(detection[2])
                        h = int(detection[3])
                        # Rectangle coordinates
                        x = int(center_x - w / 2)
                        y = int(center_y - h / 2)
                        boxes.append([x, y, w, h])
                        confidences.append(float(confidence))
                        class_ids.append(class_id)

            # # 객체 검출 결과 확인
            # if isinstance(results, torch.Tensor):  # Tensor일 경우
            #     print("객체 검출 결과가 Tensor로 반환됐습니다.")
            # elif isinstance(results, list) and all(isinstance(pred, torch.Tensor) for pred in results):  # 리스트 내부가 모두 Tensor일 경우
            #     print("객체 검출 결과가 Tensor의 리스트로 반환됐습니다.")
            # else:
            #     print("객체 검출 결과가 적절하지 않은 형식입니다.")

            # 중복 박스 삭제
            indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

            # 객체 검출 결과 처리 및 표시
            font = cv2.FONT_HERSHEY_PLAIN
            for i in range(len(boxes)):
                if i in indexes:
                    x, y, w, h = boxes[i]
                    label = str(classes[class_ids[i]])
                    cv2.rectangle(img, (x, y), (x + w, y + h), 2)
                    cv2.putText(img, label, (x, y + 30), font, 3, 3)
            cv2.imshow("Image", img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

        except Exception as e:
            print("Error:", e)
            break

    # 비디오 스트림 및 OpenCV 윈도우 종료
    cv2.destroyAllWindows()

In [ ]:
# main 함수 - 카메라 실행
if __name__ == "__main__":
    # YOLOv5 모델 경로 설정
    weights_path = "/content/best.pt"

    # 객체 검출 함수 호출
    detect_objects_yolov5(weights_path)

/usr/local/lib/python3.10/dist-packages/torch/hub.py:294: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


requirements: Ultralytics requirements ['gitpython>=3.1.30', 'pillow>=10.3.0'] not found, attempting AutoUpdate...




---
# 원본 코드


In [ ]:
# # 웹캠 활성화 되는지 확인하는 코드
# %%javascript

# async function testWebcam() {
#     const stream = await navigator.mediaDevices.getUserMedia({ video: true });
#     const video = document.createElement('video');
#     video.srcObject = stream;
#     video.play();
#     document.body.appendChild(video);
# }
# testWebcam();

In [ ]:
######################################## 원본
# # 파이썬 take_photo 함수
# def take_photo(filename='photo.jpg', quality=0.8):

#   # javascript 작성 시작
#   js = Javascript('''
#     async function takePhoto(quality) {

#       //div(공간) 생성
#       const div = document.createElement('div');

#       //capture button 생성
#       const capture = document.createElement('button');
#       capture.textContent = 'Capture';
#       div.appendChild(capture);

#       //quit button 생성
#       const quit = document.createElement('button');
#       quit.textContent = 'Quit';
#       div.appendChild(quit);
#       //quit button에 들어갈 함수 추가 - 코드 종류 함수
#       function winClose(){
#         // window.open("about:black","_self").close();
#         window.open("about:black","_self").exit();
#       }
#       //quit button에 함수 할당
#       quit.addEventListener('click', winClose);

#       //video 생성
#       const video = document.createElement('video');
#       //비디오 모양 네모네모
#       video.style.display = 'block';
#       //카메라(웹캠) 불러오기
#       const stream = await navigator.mediaDevices.getUserMedia({video: true});

#       //div 밑에 child 공간 추가
#       document.body.appendChild(div);
#       //공간에 video 넣기
#       div.appendChild(video);
#       //video와 웹캠 연결
#       video.srcObject = stream;
#       //await -> 비동기식 처리 (thread와 관련)(async와 세트)
#       await video.play();

#       // Resize the output to fit the video element.
#       google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

#       // // Wait for Capture to be clicked.
#       // await new Promise((resolve) => capture.onclick = resolve);

#       //canvas 생성
#       const canvas = document.createElement('canvas');
#       //크기 맞추기
#       canvas.width = video.videoWidth;
#       canvas.height = video.videoHeight;
#       //이미지 그리기
#       canvas.getContext('2d').drawImage(video, 0, 0);
#       //비디오 끄고, div 삭제 코드가 있어야 캡쳐 후 카메라가 여러 개 생기지 않음
#       //비디오 끄기
#       stream.getVideoTracks()[0].stop();
#       //div 삭제
#       div.remove();
#       // 파일 주소 반환
#       return canvas.toDataURL('image/jpeg', quality);

#     }
#     ''')
#   display(js)

#   # javascript 코드 실행한 결과 반환값을 파이썬에게 전달
#   data = eval_js('takePhoto({})'.format(quality))

#   #웹 브라우저에서 데이터를 저장할때 base64로 저장
#   binary = b64decode(data.split(',')[1])

#   with open(filename, 'wb') as f:
#       f.write(binary)
#   return filename

In [ ]:
# ######################################## 원본
# def detect_objects_yolov5(weights_path, img_size=416):
#     # YOLOv5 모델 로드
#     model = torch.hub.load('ultralytics/yolov5', 'custom', path=weights_path, force_reload=True)

#     # 웹캠 비디오 스트림 열기 및 객체 검출 수행
# #    while cv2.waitKey(33) != ord('q'):
#     while True:

#         try:
#             # 카메라로부터 이미지 가져오기
#             filename = take_photo()

#             # 프레임 읽기
#             frame = cv2.imread(filename)

#             # 프레임 전처리
#             frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#             img = torch.from_numpy(frame).float() / 255.0
#             img = img.permute(2, 0, 1)
#             img = img.unsqueeze(0)
#             img = img.to(model.model.device)

#             # 객체 검출
#             results = model(img)

#             # 객체 검출 결과 확인
#             if isinstance(results, torch.Tensor):  # Tensor일 경우
#                 print("객체 검출 결과가 Tensor로 반환됐습니다.")
#             elif isinstance(results, list) and all(isinstance(pred, torch.Tensor) for pred in results):  # 리스트 내부가 모두 Tensor일 경우
#                 print("객체 검출 결과가 Tensor의 리스트로 반환됐습니다.")
#             else:
#                 print("객체 검출 결과가 적절하지 않은 형식입니다.")

#             # 객체 검출 결과 처리 및 표시
#             for pred in results:
#                 if pred.shape[0] > 0:
#                     boxes = pred[:, :4]
#                     scores = pred[:, 4]
#                     class_ids = pred[:, 5]

#                     for box, score, class_id in zip(boxes, scores, class_ids):
#                         if score > 0.5:
#                             x1, y1, x2, y2 = map(int, box)
#                             label = model.model.names[int(class_id)]
#                             print(f"객체: {label}, 확률: {score:.2f}, 좌표: ({x1}, {y1}) - ({x2}, {y2})")

#                             # 객체 경계 상자 그리기
#                             cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

#                             # 클래스 이름과 확률 표시
#                             text = f"{label}: {score:.2f}"
#                             cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

#             # 이미지 출력
#             cv2_imshow(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))

#         except Exception as e:
#             print("Error:", e)
#             break

#     # 비디오 스트림 및 OpenCV 윈도우 종료
#     cv2.destroyAllWindows()

코드 실행하니까 quit 버튼 누르면 실행된 창이 닫히는데 코드가 종료되진 않네;;;

In [ ]:
# main 함수 - 카메라 실행
if __name__ == "__main__":
    # YOLOv5 모델 경로 설정
    weights_path = "/content/best.pt"

    # 객체 검출 함수 호출
    detect_objects_yolov5(weights_path)

Error: 'NoneType' object has no attribute 'split'
